[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/rodreras/geopy_minicurso/HEAD)

# First Steps: Remote Sensing + Python

Remote sensing is very useful in geology work, as well as other environmental sciences.

However, dealing with raster data in Python is not always straightforward, as you start to handle a higher number of dimensions, making understanding more challenging.

Here, we will give you an overview so you can get an initial idea and then go on to seek more content on this topic. Know that there's a universe of data and content to explore and apply to your studies.

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#Library for handling raster data
import rasterio as rio
from rasterio.plot import show, show_hist, reshape_as_raster

import earthpy.spatial as es
import earthpy.plot as ep
import earthpy as et

#To download the image
import urllib.request
import os
!pip install gdown
import gdown

In [ ]:
url_image = 'https://drive.google.com/file/d/1jbf3--R-1PY_WtGOFCu4L7Pnsz5rl-mB/view?usp=drive_link'

os.mkdir('./Dataa/')

path = "./Dataa/"  


file_name = 'landsat5_empilhada.tif'  # defining the file name

saving_path = os.path.join(path,file_name)  # creating the location to save

urllib.request.urlretrieve(url_image,saving_path)  # Downloading and saving

print("File downloaded successfully!")

In [ ]:
#Alternative way, using direct google downlaod


#As the image is large, and it's not recommended to keep it in the repository
#I saved it on my Google Drive, and through the requests library
#we are downloading it to the Data folder.

url_image = 'https://drive.google.com/file/d/1jbf3--R-1PY_WtGOFCu4L7Pnsz5rl-mB/view?usp=drive_link'

os.makedirs('./Dataaa/', exist_ok =True)

path = "./Dataaa/"  

file_name = 'landsat5_empilhada.tif'  # defining the file name

saving_path = os.path.join(path,file_name)  # creating the location to save

gdown.download(url_image,saving_path, quiet =False)  # Downloading and saving

print("File downloaded successfully!")

In [ ]:
# Here we have a raster file with 5 stacked bands.

img ='./Dataaa/landsat5_empilhada.tif'

In [ ]:
img

In [ ]:
# Opening the stacked raster

with rio.open(img) as src:
    bands = src.read()  # Reading all bands of the raster into 'bands'
    # Copying the raster image's metadata
    out_meta = src.profile


In [ ]:
# opening stacked raster
with rio.open(img) as src:
    bands = src.read()
    # copying the raster image metadata
    out_meta = src.profile

In [ ]:
# using the earthplot function to plot all the bands of the file
ep.plot_bands(bands,
              cbar=False,
              cmap='gist_earth')
plt.show()

## Making the First Compositions

An important thing! Even though the bands are from 1 to 6, when we write the code to plot, we have to remember that Python uses indices. That is, the bands will be understood from 0 to 5.

In [ ]:
# creating a grid of 2 figures
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(10,8), sharey=True)

# figure 0,0
ep.plot_rgb(bands,
            rgb=[2,3,1],
            stretch=True,
            figsize=(5, 6),
            ax=ax[0],
            title="RGB For Agriculture")

# figure 0,1
ep.plot_rgb(bands,
            rgb=[2,1,0],
            stretch=True,
            figsize=(5, 6),
            ax=ax[1],
            title="RGB True Color Image")

plt.show()

# Generating Histograms for Each Band

In [ ]:
ep.hist(bands,
        title=[f'Band-{i+1}' for i in range(1, 7)], 
        cols=3, 
        alpha=0.5, 
        figsize=(12, 10))

plt.show()

# Saving Image

In [ ]:
# stacking the bands with numpy
rgb = np.stack((bands[2], bands[1], bands[0]))

In [ ]:
# updating metadata
out_meta.update({"driver": "GTiff",
                 "height": bands.shape[1],
                 "width": bands.shape[2],
                 "count": 3,
                 })

# saving the file
with rio.open('./Dataaa/RGB_image.tif', "w", **out_meta) as dest:
   
    dest.write(rgb,  # here, we place where the image was stacked, in this case, rgb
               [1,2,3]  # here we indicate the number of bands and their order
               )

## Test!

Now that you've exported your image, open QGIS and load the image. It will open as an RGB true color composition.